In [1]:
import sys
import numpy as np
import matplotlib.pyplot as plt
import scipy.constants as cts
import pylcp
import time
import random
from tqdm import tqdm
from scipy.interpolate import interp1d,interp2d,RegularGridInterpolator
from scipy import stats
from scipy import integrate
from celluloid import Camera
from bayes_opt import BayesianOptimization
import json
import seaborn as sns
import White_class
from scipy import special
import pymongo
import Expected_model_simulation
from pymongo import MongoClient
from scipy.optimize import root_scalar


c:\Users\qmopl\anaconda3\envs\LCPtest\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
#Main variables

main_detune = 17
sideband_detune = 120
white_detune = 10
laser_on = 200000
laser_off = 400015
MOT_power = 50
v0_start=1
v0_step=1
v0_end=25
t0_start=0
t0_step=1
t0_end=1000000


Gamma = 22 # Hz to kHz, Decay rate
wavelength = 359.3e-9 # m to mm
k = 1/wavelength*2*np.pi #x_0
x0 = 1/k
t0 = 1/Gamma*1/(2*np.pi*1e6)
v0 = x0/t0
m0 = cts.hbar*t0/x0**2
a0 = x0/t0**2
F0 = cts.hbar/(x0*t0)
mass = 43*cts.value('atomic mass constant')/m0 # m_0
waist = 0.012/x0
v_max = 25.
z_max = 384.855e-3/x0
z_start = 384.855e-3/x0
omega = 2*np.pi*(cts.c/wavelength) #Transition frequency
Isat = np.pi*cts.h*cts.c*Gamma*2*np.pi*1e6/3*1/(wavelength)**3
t_eval = np.arange(t0_start,t0_end,t0_step)

# The detunings used in the PRAs:
intensities = 2.*MOT_power*1e-3/(np.pi*0.012**2)/Isat

#Define the hamiltonian
H0_X, Bq_X, U_X, Xbasis = pylcp.hamiltonians.XFmolecules.Xstate(N=1,I=0.5,B=15496.8125/Gamma,
gamma = 50.697/Gamma,b=154.7/Gamma, c=178.5/Gamma,gI=5.585,gS=2.0023193043622,
    muB = cts.value('Bohr magneton in Hz/T')/1e6*1e-4/Gamma,
    muN=cts.m_e/cts.m_p*cts.value('Bohr magneton in Hz/T')*1e-4*1e-6/Gamma,return_basis=True
    )

# b : SI coupling(isotropic), c : Iz Sz coupling(anisotropic), cc : I N coupling, gamma : S N coupling

E_X = np.unique(np.diag(H0_X))

H0_A, Bq_A, Abasis = pylcp.hamiltonians.XFmolecules.Astate(J=0.5,I=0.5,
    P=+1,B=15788.2/Gamma,D=0.,H=0.,a=109./Gamma,b=-299.2/Gamma,c=274.2/Gamma,q=0., p=15./Gamma,
    muB=cts.value('Bohr magneton in Hz/T')/1e6*1e-4/Gamma,
    muN=cts.m_e/cts.m_p*cts.value('Bohr magneton in Hz/T')*1e-4*1e-6/Gamma,
    gl=53/(2*15788.2),glprime=15/(2*15788.2),greprime=0.,return_basis=True
    )
# gJ : Lande g-factor, p : parity(e parity)


E_A = np.unique(np.diag(H0_A))

dijq = pylcp.hamiltonians.XFmolecules.dipoleXandAstates(
    Xbasis, Abasis, UX=U_X
    )

hamiltonian = pylcp.hamiltonian(H0_X, H0_A, Bq_X, Bq_A, dijq,mass = mass)

In [5]:
connection = MongoClient("localhost:27017")
db = connection.db.Expected_model
max_parameters = db.distinct(key = 'params',filter={'target' : {'$gte' : 0.01275}})

In [6]:
start = time.time()

print(Expected_model_simulation.Expected_simulation(**max_parameters[0]))

print(time.time()-start)

0.012984867292097274
149.85220074653625


In [4]:
pbounds = {'main_det':(10,20),'det_1':(70,170),'det_2':(5,30),'beta_1':(0,5),'beta_2':(0,15),'laseron':(138240,300000),'laseroff':(300001,600000)}

In [5]:
optimizer = BayesianOptimization(
    f = Expected_model_simulation.Expected_simulation,
    pbounds=pbounds,
    verbose=13,
    random_state=1,
)

In [6]:
from bayes_opt.util import load_logs

# New optimizer is loaded with previously seen points
load_logs(optimizer, logs=["./logs_Exp_multi.json"])

In [7]:
start = time.time()
optimizer.maximize(
    init_points=1,
    n_iter=0,)
print(time.time()-start)

|   iter    |  target   |  beta_1   |  beta_2   |   det_1   |   det_2   | laseroff  |  laseron  | main_det  |
-------------------------------------------------------------------------------------------------------------
|  1        |  0.0      |  2.085    |  10.8     |  70.01    |  12.56    |  3.44e+05 |  1.532e+0 |  11.86    |
260.05457377433777


In [8]:
from bayes_opt.logger import JSONLogger
from bayes_opt.event import Events

logger = JSONLogger(path='./logs_Exp_multi.json')
optimizer.subscribe(Events.OPTIMIZATION_STEP,logger)

In [9]:
start = time.time()
optimizer.maximize(
    init_points=5,
    n_iter=10,)
print(time.time()-start)

|   iter    |  target   |  beta_1   |  beta_2   |   det_1   |   det_2   | laseroff  |  laseron  | main_det  |
-------------------------------------------------------------------------------------------------------------
|  2        |  0.0      |  1.728    |  5.952    |  123.9    |  15.48    |  5.056e+0 |  1.713e+0 |  18.78    |
|  3        |  0.004483 |  0.1369   |  10.06    |  111.7    |  18.97    |  3.421e+0 |  1.703e+0 |  18.01    |
|  4        |  0.0      |  4.841    |  4.701    |  139.2    |  26.91    |  5.684e+0 |  1.52e+05 |  10.39    |
|  5        |  0.0      |  0.8492   |  13.17    |  79.83    |  15.53    |  5.874e+0 |  2.245e+0 |  16.92    |
|  6        |  0.0      |  1.578    |  10.3     |  153.5    |  5.457    |  5.25e+05 |  2.982e+0 |  17.48    |
|  7        |  0.0      |  1.12     |  14.5     |  123.4    |  23.71    |  5.496e+0 |  1.88e+05 |  15.54    |
|  8        |  0.000165 |  4.996    |  0.4568   |  70.01    |  19.5     |  4.122e+0 |  2.345e+0 |  19.1     |
|  9      

In [10]:
start = time.time()
optimizer.maximize(
    init_points=0,
    n_iter=30,)
print(time.time()-start)

|   iter    |  target   |  beta_1   |  beta_2   |   det_1   |   det_2   | laseroff  |  laseron  | main_det  |
-------------------------------------------------------------------------------------------------------------
|  17       |  0.0      |  4.13     |  11.66    |  141.7    |  23.78    |  5.576e+0 |  1.393e+0 |  17.07    |
|  18       |  0.000143 |  4.139    |  8.871    |  131.4    |  18.73    |  4.794e+0 |  2.415e+0 |  13.45    |
|  19       |  0.0      |  0.685    |  14.9     |  100.1    |  21.92    |  3.38e+05 |  2.55e+05 |  10.79    |
|  20       |  0.001354 |  3.0      |  3.875    |  155.1    |  18.3     |  4.011e+0 |  2.791e+0 |  16.07    |
|  21       |  0.0      |  0.09493  |  12.94    |  112.7    |  9.89     |  5.044e+0 |  2.052e+0 |  13.8     |
|  22       |  0.0      |  1.186    |  7.141    |  98.22    |  28.7     |  5.705e+0 |  2.784e+0 |  18.14    |
|  23       |  0.00108  |  2.27     |  13.07    |  114.7    |  7.129    |  3.206e+0 |  2.561e+0 |  17.09    |
|  24     

In [11]:
optimizer.maximize(n_iter=1,init_points = 0)

|   iter    |  target   |  beta_1   |  beta_2   |   det_1   |   det_2   | laseroff  |  laseron  | main_det  |
-------------------------------------------------------------------------------------------------------------
|  47       |  0.005689 |  2.149    |  0.3916   |  103.7    |  19.79    |  4.88e+05 |  2.369e+0 |  19.61    |


In [17]:
start = time.time()
optimizer.maximize(n_iter=5,init_points = 45)

print(time.time()-start)

|   iter    |  target   |  beta_1   |  beta_2   |   det_1   |   det_2   | laseroff  |  laseron  | main_det  |
-------------------------------------------------------------------------------------------------------------
|  148      |  0.0      |  2.016    |  11.38    |  141.7    |  29.68    |  3.834e+0 |  1.389e+0 |  19.34    |
|  149      |  0.003363 |  4.289    |  10.93    |  121.7    |  22.67    |  5.342e+0 |  1.989e+0 |  17.7     |
|  150      |  0.0      |  3.753    |  9.198    |  110.2    |  22.43    |  3.009e+0 |  2.636e+0 |  18.96    |
|  151      |  0.0      |  1.197    |  1.812    |  92.03    |  12.55    |  5.649e+0 |  2.261e+0 |  12.87    |
|  152      |  0.01042  |  0.6918   |  4.352    |  131.4    |  13.1     |  4.372e+0 |  2.101e+0 |  18.28    |
|  153      |  0.00309  |  2.132    |  5.185    |  137.5    |  10.54    |  4.402e+0 |  1.892e+0 |  16.27    |
|  154      |  0.000265 |  4.387    |  6.715    |  148.4    |  16.42    |  4.969e+0 |  1.596e+0 |  14.33    |
|  155    

In [18]:
optimizer.max

{'target': 0.012751073397031026,
 'params': {'beta_1': 2.935124145664613,
  'beta_2': 10.312814186442964,
  'det_1': 115.49496654453418,
  'det_2': 15.388812962322799,
  'laseroff': 411581.2063215908,
  'laseron': 235170.32507489598,
  'main_det': 19.67630442684696}}

In [36]:
rest = db.find(projection={"_id" : 0}).sort("target",-1).limit(3)

In [37]:
for res in rest:
    print(res['params'])


{'beta_1': 2.935124145664613, 'beta_2': 10.312814186442964, 'det_1': 115.49496654453418, 'det_2': 15.388812962322799, 'laseroff': 411581.2063215908, 'laseron': 235170.32507489598, 'main_det': 19.67630442684696}
{'beta_1': 2.704759024810943, 'beta_2': 14.391514815926286, 'det_1': 150.39608914986468, 'det_2': 5.80807666558935, 'laseroff': 512816.4659821373, 'laseron': 213458.63965771275, 'main_det': 19.475489413950996}
{'beta_1': 2.704759024810943, 'beta_2': 14.391514815926286, 'det_1': 150.39608914986468, 'det_2': 5.80807666558935, 'laseroff': 512816.4659821373, 'laseron': 213458.63965771275, 'main_det': 19.475489413950996}
